In [1]:
import pandas as pd
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
collection = db.companies

In [3]:
# FILTER {$and: [ {deadpooled_year:null}, {founded_year: {$gt:2003}}, {offices: { $ne: [] } }, {$or: [{category_code:'games_video'},{category_code:'software'}] }]}
# PROJECT {name:1 , category_code:1, number_of_employees:1, founded_year:1, "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
# SORT {number_of_employees:-1}

In [4]:
def query_to_db():
    cursor = collection.find({'$and': 
                        [ {'deadpooled_year':None}, 
                          {'founded_year': {'$gt':2003}},
                          {'offices': { '$ne': [] } },
                          {'$or': [{'category_code':'games_video'},
                                   {'category_code':'software'}] 
                        }]}
                        , {'name':1 , 'category_code':1, 'number_of_employees':1, 'founded_year':1, 
                           "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
                        ).sort('number_of_employees', -1)
    return pd.DataFrame(cursor)

In [5]:
def get_columns(data):
    data['country']= data['offices'].apply(lambda elem : elem[0]['country_code'])
    data['city']= data['offices'].apply(lambda elem : elem[0]['city'])
    data['lat']= data['offices'].apply(lambda elem : elem[0]['latitude'])
    data['long']= data['offices'].apply(lambda elem : elem[0]['longitude'])
    return data
    


In [6]:
def clean_cols_nans(data):
    data= data.drop(['_id', 'offices'], axis=1)
    data= data.dropna(subset=['lat', 'long'])
    return data

In [7]:
def calculate_type(reg):
    if reg['founded_year'] > 2007 and reg['number_of_employees'] < 50:
        return 'Startup'
    else: return 'BigCompany'
    
def get_class(data):
    data['type']= data.apply(calculate_type, axis=1)
    return data

In [8]:
# Main
df= query_to_db()
df_selected= get_columns(df)
df_clean= clean_cols_nans(df_selected)
df_final= get_class(df_clean)

In [9]:
df_final[500:550]

,category_code,founded_year,name,number_of_employees,country,city,lat,long,type
715,software,2012,Carfeine,5.0,USA,Bethesda,38.989124,-77.026676,Startup
716,software,2007,Syntac Ventures,5.0,AUS,Melbourne,-37.813339,144.970823,BigCompany
718,software,2007,ORELIA,5.0,FRA,Fontainebleau,48.407483,2.701038,BigCompany
719,software,2005,Blue Sky Technologies,5.0,USA,Coppell,32.984488,-96.993293,BigCompany
720,software,2008,Uprizer Labs,5.0,USA,Austin,30.291391,-97.710481,Startup
721,software,2006,TimeLapse mobile,5.0,USA,Novato,38.068295,-122.532658,BigCompany
722,software,2007,ipsojobs,5.0,ESP,Barcelona,41.406248,2.163194,BigCompany
723,software,2008,TweetDeck,5.0,GBR,London,51.500152,-0.126236,Startup
724,games_video,2007,FABINET,5.0,DEU,Leipzig,51.318503,12.381492,BigCompany
725,software,2005,Instil Software,5.0,GBR,Belfast,54.597233,-5.879080,BigCompany


In [209]:
df_final.to_json('data_to_plot_mongo.json', orient='records')